# Sentiment Analysis Poltik Indonesia

# Model Training & Evaluation
Model yang digunakan untuk training dataset hasil dari TF-IDF adalah traditional mechine learning <br>
karena dataset yang dimiliki relatif kecil

In [1]:
# Load Dataset
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv('rev_tweet_preprocess_tfidf.csv')
df.head()

,sentimen,tweet,tweet_tokenized,tweet_no_stopwords,tweet_stemmed,tweet_process
0,negatif,kata indonesia tidak dihargai bangsa asing ...,"['kata', '@prabowo', 'indonesia', 'tidak', 'di...","['@prabowo', 'indonesia', 'dihargai', 'bangsa'...","['prabowo', 'indonesia', 'harga', 'bangsa', 'a...",prabowo indonesia harga bangsa asing berita ...
1,netral,batuan langka tasbih jokowi hadiah dari habib ...,"['batuan', 'langka', ',', 'tasbih', 'jokowi', ...","['batuan', 'langka', ',', 'tasbih', 'jokowi', ...","['batu', 'langka', '', 'tasbih', 'jokowi', 'ha...",batu langka tasbih jokowi hadiah habib luthfi...
2,netral,di era jokowi ekonomi indonesia semakin baik i...,"['di', 'era', 'jokowi', ',', 'ekonomi', 'indon...","['era', 'jokowi', ',', 'ekonomi', 'indonesia',...","['era', 'jokowi', '', 'ekonomi', 'indonesia', ...",era jokowi ekonomi indonesia 01indonesiamaju...
3,positif,bagi sumatera selatan asian games berdampak pd...,"['bagi', 'sumatera', 'selatan', ',', 'asian', ...","['sumatera', 'selatan', ',', 'asian', 'games',...","['sumatera', 'selatan', '', 'asi', 'games', 'd...",sumatera selatan asi games dampak pd ekonomi ...
4,negatif,negara kita ngutang buat bngun infrastruktur y...,"['negara', 'kita', 'ngutang', 'buat', 'bngun',...","['negara', 'ngutang', 'bngun', 'infrastruktur'...","['negara', 'ngutang', 'bngun', 'infrastruktur'...",negara ngutang bngun infrastruktur udah dipake...


In [2]:
# Buat variabel label fitur dan target
X_text = df['tweet_process'].fillna('')
y = df['sentimen']

vectorizer = TfidfVectorizer(max_features=3000)
X = vectorizer.fit_transform(X_text)
print('Shape TF-IDF matrix:', X.shape)

Shape TF-IDF matrix: (1815, 3000)


In [3]:
# Split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
print('Jumlah data train : ', X_train.shape[0])
print('Jumlah data test : ', X_test.shape[0])

Jumlah data train :  1452
Jumlah data test :  363


- Training model menggunakan Logistic Regression <br>
(cocok untuk klasifikasi biner multikelas)

In [4]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

- Evaluasi model

In [5]:
from sklearn.metrics import accuracy_score, classification_report
y_pred = model.predict(X_test)
print("Akurasi:", accuracy_score(y_test, y_pred))
print("Laporan Klasifikasi:\n", classification_report(y_test, y_pred))

Akurasi: 0.6253443526170799
Laporan Klasifikasi:
               precision    recall  f1-score   support

     negatif       0.63      0.74      0.68       117
      netral       0.64      0.53      0.58       127
     positif       0.60      0.62      0.61       119

    accuracy                           0.63       363
   macro avg       0.63      0.63      0.62       363
weighted avg       0.63      0.63      0.62       363



- Hyperparameter Tuning menggunakan GridSearch

In [6]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100], 
    'solver': ['lbfgs', 'liblinear'], 
    'penalty': ['l2'],
    'class_weight': ['balanced', None]
    }
grid = GridSearchCV(LogisticRegression(max_iter=3000), param_grid, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)
print("Parameter terbaik:", grid.best_params_)
print("Akurasi terbaik:", grid.best_score_)

Parameter terbaik: {'C': 1, 'class_weight': None, 'penalty': 'l2', 'solver': 'liblinear'}
Akurasi terbaik: 0.6294726863372437


- coba algoritma lain
- count factorizer (binary)
- coba tweet tokenizer
- coba bandingan data yang ada isi dan tanpa hastag
- maksimal 0.70 sekarang
- lihat score klasifikasi, identifikasi kenapa ada score yang kecil 
- cek untuk deep learning
- tweet_process itu kan digabungkan jadi 1 kalimat, coba untuk tetap di split per kata untuk lihat akurasinya


- Simpan model 

In [7]:
---

SyntaxError: invalid syntax (1947214667.py, line 1)